#**torch.profiler**

O pacote torch.profiler no PyTorch é uma ferramenta que permite a coleta de métricas de desempenho durante o treinamento e a inferência de modelos de aprendizado de máquina. Com a API de gerenciamento de contexto do torch.profiler, você pode entender quais operadores do modelo são os mais custosos, examinar suas formas de entrada e rastreamentos de pilha, estudar a atividade do kernel do dispositivo e visualizar o rastro de execução

In [ ]:
%%capture
!pip install torch_tb_profiler

In [ ]:
import torch
import torchvision
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

##**Criando o modelo ResNet e gerando dados aleatórios**

In [ ]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

##**Fazendo uma predição com base nos dados aleatórios e acompanhando a execução de forma simples**

In [ ]:
with profile(activities=[ProfilerActivity.CPU],
             record_shapes=True) as model_profile:
    with record_function("model_inference"):
        model(inputs)

##**imprimindo os resultados**

In [ ]:
print(model_profile.key_averages().table())

In [ ]:
print(model_profile.key_averages().table(sort_by="cpu_time_total",
                                         row_limit=10))

In [ ]:
print(model_profile.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total",
                                                                  row_limit=10))

##**Acompanhando execução em CPU e GPU**

Uma maneira de usar o torch.profiler para coletar informações detalhadas sobre o desempenho de um modelo PyTorch durante a inferência

- with profile(...) as model_profile: Este bloco de código inicia o contexto do profiler, especificando quais atividades devem ser monitoradas. Neste caso, estamos monitorando atividades na CPU e na CUDA (GPU).
- activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]: Define as atividades que serão perfiladas. ProfilerActivity.CPU monitora operações na CPU, enquanto ProfilerActivity.CUDA monitora operações na GPU.
- record_shapes=True: Indica que o profiler deve registrar as formas (shapes) dos tensores que são passados para as operações.
- as model_profile: Este é o objeto de perfil que será usado para acessar os dados coletados após a execução do bloco.
- with record_function("model_inference"): Um contexto que permite rotular um bloco de código específico, neste caso, a inferência do modelo. Isso ajuda a identificar e separar as métricas de desempenho para essa parte específica do código.
- model(inputs): É a chamada ao modelo com os dados de entrada, que é o código que você deseja perfilar.

Após a execução desse bloco de código, o objeto model_profile conterá os dados de desempenho coletados, que possasmos analisar para entender onde estão os gargalos de desempenho e como otimizar o modelo.



In [ ]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[
        ProfilerActivity.CPU,
        ProfilerActivity.CUDA
        ],
             record_shapes=True) as model_profile:
    with record_function("model_inference"):
        model(inputs)

print(model_profile.key_averages().table(sort_by="cuda_time_total",
                                         row_limit=10))

In [ ]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU],
             profile_memory=True, #rastreia alocação e liberação de memória
             record_shapes=True) as model_profile:
    model(inputs)

In [ ]:
print(model_profile.key_averages().table(sort_by="cpu_memory_usage",
                                         row_limit=10))

In [ ]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[ProfilerActivity.CPU,
                         ProfilerActivity.CUDA]) as model_profile:
    model(inputs)

model_profile.export_chrome_trace("trace.json")

In [ ]:
with profile(
    activities=[ProfilerActivity.CPU,
                ProfilerActivity.CUDA],
    with_stack=True, #O parâmetro with_stack=True é usado no contexto do torch.profiler d
                     #o PyTorch para incluir informações de rastreamento de pilha nos dados de perfil.
                     #Quando ativado, ele permite que o profiler registre a pilha de chamadas que levou
                     #a cada operação registrada. Isso é útil para identificar exatamente onde no código
                     #fonte as operações mais custosas estão sendo chamadas1
) as model_profile:
    model(inputs)

print(model_profile.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

In [ ]:
model_profile.export_stacks("/tmp/profiler_stacks.txt",
                            "self_cuda_time_total")

In [ ]:
import os
os.listdir('/tmp')

##**Adicionando um Schedule para customizar a coleta de dados**

- from torch.profiler import schedule: Esta linha importa a função schedule do módulo torch.profiler.
- my_schedule = schedule(...): Aqui, my_schedule é definido como uma chamada à função schedule, que retorna um objeto de agendamento configurável.
- skip_first=10: O profiler irá ignorar as primeiras 10 etapas (ou iterações) antes de começar qualquer coleta de dados. Isso pode ser útil para evitar a medição de tempos de inicialização atípicos.
- wait=5: Após pular as etapas iniciais, o profiler irá esperar por mais 5 etapas sem coletar dados. Isso permite que o modelo estabilize e reduz o impacto do overhead de inicialização.
- warmup=1: Durante a fase de aquecimento, que dura 1 etapa, o profiler começa a preparar-se para a coleta de dados, mas ainda não coleta métricas ativamente.
- active=3: Após o aquecimento, o profiler ficará ativo e coletará dados por 3 etapas.
- repeat=2: Todo o ciclo de wait, warmup e active será repetido 2 vezes.

In [ ]:
from torch.profiler import schedule

my_schedule = schedule(
    skip_first=10,
    wait=5,
    warmup=1,
    active=3,
    repeat=2)

A função trace_handler é um manipulador de rastreamento personalizado usado com o torch.profiler para processar os dados de perfil coletados durante a execução do modelo.

- model_profile: Este é o objeto que contém os dados de perfil coletados pelo torch.profiler.
- model_profile.key_averages(): Calcula as médias das métricas de desempenho para cada operador ou evento que foi rastreado.
.table(sort_by="self_cuda_time_total", row_limit=10): Gera uma tabela formatada das médias, ordenando os resultados pelo tempo total gasto em operações CUDA (self_cuda_time_total) e limitando a saída às 10 linhas superiores.
- print(output): Imprime a tabela formatada no console, permitindo que você veja rapidamente quais operadores ou eventos estão consumindo mais tempo na GPU.
- model_profile.export_chrome_trace(...): Exporta os dados de perfil para um arquivo JSON que pode ser visualizado usando o chrome://tracing no navegador Google Chrome. O nome do arquivo inclui o número da etapa de perfil (step_num), que ajuda a identificar e organizar os arquivos de rastreamento se você estiver coletando vários ao longo do tempo.


In [ ]:
def trace_handler(model_profile):
    output = model_profile.key_averages().table(sort_by="self_cuda_time_total",
                                                row_limit=10)
    print(output)
    model_profile.export_chrome_trace("/tmp/trace_" + str(model_profile.step_num) + ".json")

In [ ]:
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=2),
    on_trace_ready=trace_handler
) as model_profile:
    for idx in range(8):
        model(inputs)
        model_profile.step()

#**Aplicando o torch.profiler em alguns passos do treino da ResNet**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = torchvision.models.resnet18(weights='IMAGENET1K_V1').to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
model.train()

##**Fazendo os imports**

In [ ]:
import torch
import torch.nn
import torch.optim
import torch.profiler
import torch.utils.data
import torchvision.datasets
import torchvision.models
import torchvision.transforms as T

##**Criando o processamento do dado dataset e dataloader**

In [ ]:
transform = T.Compose(
    [T.Resize(224),
     T.ToTensor(),
     T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data',
                                         train=True,
                                         download=True,
                                         transform=transform)

train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=32,
                                           shuffle=True)

## **Criando a função que executa um passo no treinamento**

In [ ]:
def train_step(data):
    inputs, labels = data[0].to(device=device), data[1].to(device=device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
scheduler = torch.profiler.schedule(wait=1,
                                   warmup=1,
                                   active=3,
                                   repeat=1)

In [ ]:
with torch.profiler.profile(
        schedule=scheduler,
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/resnet18'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
) as model_profile:

    for step, batch_data in enumerate(train_loader):
        model_profile.step()
        if step == 10:
            break
        train_step(batch_data)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./log/resnet18